In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-25 02:24:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.79MB/s    in 0.2s    

2021-05-25 02:24:37 (4.79 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [129]:
#select_sql = "(SELECT points, variety, avg_temp, avg_airp, avg_humid, avg_precip FROM machine"
wine_rds = spark.read.jdbc(url=jdbc_url, table='weather_mapscaled', properties=config)
#county_df = spark.read.jdbc(url=jdbc_url, table='counties', properties=config)
wine_rds.show()

+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|  winery_search_term|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|     country|points|           variety|              winery|
+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|   LaZarre winery US| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|          US|    88|        Chardonnay|             LaZarre|
|Negretti winery I...| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|       Italy|    88|        Chardonnay|            Negretti|
|Pico Maccario win...| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|     

In [130]:
wine_rds.columns

['winery_search_term',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip',
 'country',
 'points',
 'variety',
 'winery']

In [131]:
wine_ids = wine_rds.select('country',
 'points',
 'variety',
 'winery')
wine_ids.dtypes

[('country', 'string'),
 ('points', 'int'),
 ('variety', 'string'),
 ('winery', 'string')]

In [132]:
allwines = pd.DataFrame(wine_rds.select('lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip','points','variety').collect(), columns=['lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip','points','variety'], dtype=float)
allwines.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
points        float64
variety        object
dtype: object

In [184]:
uscounties = spark.read.jdbc(url=jdbc_url, table='uscounty', properties=config)
uscounties.show(5)

+----+---+---+---------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
| zip|lat|lng|     city|state_id|   state_name|county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|
+----+---+---+---------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
|1001| 42|-73|   Agawam|      MA|Massachusetts|    Hampden|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1002| 42|-72|  Amherst|      MA|Massachusetts|  Hampshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1005| 42|-72|    Barre|      MA|Massachusetts|  Worcester|270.7250000000000...|1014.960000000000...|71.75250000000000...|0.135000000000000000|
|1029| 42|-73|East Otis|      MA|Massachusetts|  Berkshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000

In [185]:
uscounties.columns

['zip',
 'lat',
 'lng',
 'city',
 'state_id',
 'state_name',
 'county_name',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip']

In [186]:
county_ids = pd.DataFrame((uscounties.select('zip',
                                             'city',
                                             'state_id',
                                             'state_name',
                                             'county_name').collect()), columns=['zip',
                                             'city',
                                             'state_id',
                                             'state_name',
                                             'county_name']).dropna()
county_ids.head(4)

,zip,city,state_id,state_name,county_name
0,1001,Agawam,MA,Massachusetts,Hampden
1,1002,Amherst,MA,Massachusetts,Hampshire
2,1005,Barre,MA,Massachusetts,Worcester
3,1029,East Otis,MA,Massachusetts,Berkshire


In [187]:
len(county_ids)

1797

In [188]:
bigcounty = pd.DataFrame((uscounties.select('zip',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip').collect()), columns=['zip',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip'], dtype=float).dropna()
bigcounty.dtypes

zip           float64
lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
dtype: object

In [189]:
len(bigcounty)

1795

In [190]:
countydrops = ['zip']

In [191]:
allwines['variety'].unique()

array(['Chardonnay', 'Sauvignon Blanc', 'Rosé', 'Cabernet Sauvignon',
       'Merlot', 'Pinot Noir', 'Riesling', 'Syrah'], dtype=object)

In [192]:
mer_df = allwines[allwines['variety'] == 'Merlot']
syr_df = allwines[allwines['variety'] == 'Syrah']
savyb_df = allwines[allwines['variety'] == 'Sauvignon Blanc']
ries_df = allwines[allwines['variety'] == 'Riesling']
cabsav_df = allwines[allwines['variety'] == 'Cabernet Sauvignon']
char_df = allwines[allwines['variety'] == 'Chardonnay']
pino_df = allwines[allwines['variety'] == 'Pinot Noir']

In [193]:
mer_df.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
points        float64
variety        object
dtype: object

In [194]:
winedropcols = ['points','variety']

In [195]:
#features and targets
variety = mer_df.copy()
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.31708703561405066

In [196]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
mer_preds = rf.predict(Xcount)

In [197]:
#features and targets
variety = syr_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.24766239118394037

In [198]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
syr_preds = rf.predict(Xcount)

In [199]:
#features and targets
variety = savyb_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.2740873924253874

In [200]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
savyb_preds = rf.predict(Xcount)

In [201]:
#features and targets
variety = ries_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.32150114413761166

In [202]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
ries_preds = rf.predict(Xcount)

In [203]:
#features and targets
variety = cabsav_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.2602562254348104

In [204]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
cabsav_preds = rf.predict(Xcount)

In [205]:
#features and targets
variety = char_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.281147493665847

In [206]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
char_preds = rf.predict(Xcount)

In [207]:
#features and targets
variety = pino_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.1950767218357803

In [208]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
pino_preds = rf.predict(Xcount)

In [209]:
counties_preds = bigcounty.copy()
counties_preds['mer_points'] = mer_preds
counties_preds['syr_points'] = syr_preds
counties_preds['savyb_points'] = savyb_preds
counties_preds['ries_points'] = ries_preds
counties_preds['cabsav_points'] = cabsav_preds
counties_preds['char_points'] = char_preds
counties_preds['pinot_points'] = pino_preds

In [210]:
counties_preds.describe()

,zip,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip,mer_points,syr_points,savyb_points,ries_points,cabsav_points,char_points,pinot_points
count,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000
mean,50734.467967,38.357103,-91.311421,274.303437,1017.654591,74.131337,0.106546,86.002639,87.398963,86.565681,86.920057,87.268075,86.668671,87.569186
std,24597.722869,5.101211,12.403000,6.620178,4.407807,6.428987,0.098159,1.417427,1.602512,1.352219,1.070443,1.742589,1.592718,1.696042
min,1001.000000,26.000000,-124.000000,260.640000,962.630000,37.947500,0.000000,82.256850,82.567656,82.433153,82.013727,82.452449,82.193786,82.812724
25%,30127.000000,34.000000,-99.000000,269.471250,1016.560000,71.022500,0.017500,85.072781,86.482048,85.666644,86.204160,86.046166,85.667427,86.443757
50%,50515.000000,38.000000,-89.000000,274.070000,1019.040000,75.952500,0.097500,86.017360,87.446114,86.602199,87.004087,87.250444,86.617376,87.418991
75%,71632.500000,42.000000,-82.000000,279.122500,1020.105000,78.457500,0.155000,87.017417,88.177460,87.447183,87.580843,88.370631,87.542553,88.553000
max,99401.000000,49.000000,-68.000000,295.845000,1022.525000,87.015000,0.917500,91.516494,93.027501,90.791276,90.845201,93.909851,92.952938,93.428774


In [211]:
savyb_df.describe()

,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip,points
count,4718.000000,4718.000000,4718.000000,4718.000000,4718.000000,4718.000000,4718.000000
mean,20.845485,-47.642433,282.140502,1014.725774,75.244232,0.096221,87.454854
std,33.964335,87.643879,6.913607,9.921554,11.133844,0.081495,2.681127
min,-49.000000,-124.000000,263.640000,942.702500,35.680000,0.000000,80.000000
25%,-33.000000,-121.000000,278.405000,1012.035000,71.685000,0.057500,86.000000
50%,38.000000,-77.000000,281.032500,1018.500000,78.512500,0.077500,87.000000
75%,43.000000,3.000000,289.522500,1019.670000,80.530000,0.145000,89.000000
max,53.000000,178.000000,300.137500,1025.890000,90.847500,0.917500,96.000000


In [212]:
county_output = county_ids.merge(counties_preds, how='left', on='zip')
county_output.describe()

,zip,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip,mer_points,syr_points,savyb_points,ries_points,cabsav_points,char_points,pinot_points
count,1797.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000
mean,50714.359488,38.357103,-91.311421,274.303437,1017.654591,74.131337,0.106546,86.002639,87.398963,86.565681,86.920057,87.268075,86.668671,87.569186
std,24597.447200,5.101211,12.403000,6.620178,4.407807,6.428987,0.098159,1.417427,1.602512,1.352219,1.070443,1.742589,1.592718,1.696042
min,1001.000000,26.000000,-124.000000,260.640000,962.630000,37.947500,0.000000,82.256850,82.567656,82.433153,82.013727,82.452449,82.193786,82.812724
25%,30122.000000,34.000000,-99.000000,269.471250,1016.560000,71.022500,0.017500,85.072781,86.482048,85.666644,86.204160,86.046166,85.667427,86.443757
50%,50451.000000,38.000000,-89.000000,274.070000,1019.040000,75.952500,0.097500,86.017360,87.446114,86.602199,87.004087,87.250444,86.617376,87.418991
75%,71630.000000,42.000000,-82.000000,279.122500,1020.105000,78.457500,0.155000,87.017417,88.177460,87.447183,87.580843,88.370631,87.542553,88.553000
max,99401.000000,49.000000,-68.000000,295.845000,1022.525000,87.015000,0.917500,91.516494,93.027501,90.791276,90.845201,93.909851,92.952938,93.428774


In [213]:
#counties_preds.to_csv('county_predictions.csv')
#counties_preds.to_json('county_predictions.json')

In [214]:
#spark_counties = spark.createDataFrame(counties_preds)
#spark_counties.show()

In [215]:
#spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [216]:
#sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [217]:
#sparked_preds.show()

In [218]:
#winners_df = pd.DataFrame(sparked_preds.select('zip','lat','lng','state_id','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success').collect(), columns=['zip','lat','lng','state','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success'], dtype=float)
#winners_df.describe()

In [219]:
#winners_df.columns

In [220]:
#winners_df

In [221]:
#winners_df['mer_success'].value_counts()